In [54]:
import re
import os
import random
import sys
sys.path.append('/Users/seunghoonchoi/Downloads/Dacon_recommend_system')
import requests

import warnings
warnings.filterwarnings(action='ignore') 

import ast

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from gensim.models import FastText

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import IterableDataset

import dgl
import dgl.nn as dglnn
import dgl.function as fn
from dgl.nn import HeteroGraphConv, SAGEConv

In [2]:
CFG = {'SEED' : 42}

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

In [4]:
def recall5(answer_df, submission_df):
    """
    Calculate recall@5 for given dataframes.
    
    Parameters:
    - answer_df: DataFrame containing the ground truth
    - submission_df: DataFrame containing the predictions
    
    Returns:
    - recall: Recall@5 value
    """
    
    primary_col = answer_df.columns[0]
    secondary_col = answer_df.columns[1]
    
    # Check if each primary_col entry has exactly 5 secondary_col predictions
    prediction_counts = submission_df.groupby(primary_col).size()
    if not all(prediction_counts == 5):
        raise ValueError(f"Each {primary_col} should have exactly 5 {secondary_col} predictions.")


    # Check for NULL values in the predicted secondary_col
    if submission_df[secondary_col].isnull().any():
        raise ValueError(f"Predicted {secondary_col} contains NULL values.")
    
    # Check for duplicates in the predicted secondary_col for each primary_col
    duplicated_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].duplicated().any())
    if duplicated_preds.any():
        raise ValueError(f"Predicted {secondary_col} contains duplicates for some {primary_col}.")


    # Filter the submission dataframe based on the primary_col present in the answer dataframe
    submission_df = submission_df[submission_df[primary_col].isin(answer_df[primary_col])]
    
    # For each primary_col, get the top 5 predicted secondary_col values
    top_5_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].head(5).tolist()).to_dict()
    
    # Convert the answer_df to a dictionary for easier lookup
    true_dict = answer_df.groupby(primary_col).apply(lambda x: x[secondary_col].tolist()).to_dict()
    
    
    individual_recalls = []
    for key, val in true_dict.items():
        if key in top_5_preds:
            correct_matches = len(set(true_dict[key]) & set(top_5_preds[key]))
            individual_recall = correct_matches / min(len(val), 5) # 공정한 평가를 가능하게 위하여 분모(k)를 'min(len(val), 5)' 로 설정함 
            individual_recalls.append(individual_recall)


    recall = np.mean(individual_recalls)
    return recall

In [5]:
# 이력서 관련
resume = pd.read_csv('/Users/seunghoonchoi/Downloads/Dacon_recommend_system/Data/resume.csv')
resume_edu = pd.read_csv('/Users/seunghoonchoi/Downloads/Dacon_recommend_system/Data/resume_education.csv')
resume_cert = pd.read_csv('/Users/seunghoonchoi/Downloads/Dacon_recommend_system/Data/resume_certificate.csv')
resume_lang = pd.read_csv('/Users/seunghoonchoi/Downloads/Dacon_recommend_system/Data/resume_language.csv')

In [6]:
# 공고 관련
recruitment = pd.read_csv('/Users/seunghoonchoi/Downloads/Dacon_recommend_system/Data/recruitment.csv')
company = pd.read_csv('/Users/seunghoonchoi/Downloads/Dacon_recommend_system/Data/company.csv')

In [7]:
# 이력서 - 공고 매칭
apply_train = pd.read_csv('/Users/seunghoonchoi/Downloads/Dacon_recommend_system/Data/apply_train.csv')

In [8]:
resume.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8482 entries, 0 to 8481
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resume_seq       8482 non-null   object 
 1   reg_date         8482 non-null   object 
 2   updated_date     8482 non-null   object 
 3   degree           8482 non-null   int64  
 4   graduate_date    8482 non-null   int64  
 5   hope_salary      8482 non-null   float64
 6   last_salary      8482 non-null   float64
 7   text_keyword     8287 non-null   object 
 8   job_code_seq1    8482 non-null   object 
 9   job_code_seq2    500 non-null    object 
 10  job_code_seq3    187 non-null    object 
 11  career_month     8482 non-null   int64  
 12  career_job_code  7724 non-null   object 
dtypes: float64(2), int64(3), object(8)
memory usage: 861.6+ KB


In [9]:
resume_edu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8482 entries, 0 to 8481
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   resume_seq             8482 non-null   object 
 1   hischool_type_seq      8482 non-null   int64  
 2   hischool_special_type  8482 non-null   object 
 3   hischool_nation        8482 non-null   object 
 4   hischool_gender        8482 non-null   object 
 5   hischool_location_seq  8482 non-null   int64  
 6   univ_type_seq1         8482 non-null   int64  
 7   univ_type_seq2         8482 non-null   int64  
 8   univ_transfer          8482 non-null   int64  
 9   univ_location          8482 non-null   int64  
 10  univ_major             1795 non-null   object 
 11  univ_sub_major         416 non-null    object 
 12  univ_major_type        8482 non-null   int64  
 13  univ_score             8482 non-null   float64
dtypes: float64(1), int64(7), object(6)
memory usage: 927.8+ 

In [10]:
resume_cert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12975 entries, 0 to 12974
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   resume_seq            12975 non-null  object
 1   certificate_contents  12971 non-null  object
dtypes: object(2)
memory usage: 202.9+ KB


In [11]:
resume_lang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   resume_seq  869 non-null    object 
 1   language    869 non-null    int64  
 2   exam_name   869 non-null    int64  
 3   score       869 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 27.3+ KB


In [12]:
resume_lang['language'].unique()

array([2, 4, 3, 9, 8])

In [13]:
resume_lang['exam_name'].unique()

array([ 4,  5,  6,  1, 11,  3,  9,  2, 21, 10, 24,  8, 13,  7, 31])

In [14]:
# resume를 제외한 이력서 관련 데이터프레임에 'resume_seq' 중복값 있는지 확인
resume_edu_duplicates = resume_edu['resume_seq'].value_counts()
resume_cert_duplicates = resume_cert['resume_seq'].value_counts()
resume_lang_duplicates = resume_lang['resume_seq'].value_counts()

resume_edu_duplicates_count = resume_edu_duplicates[resume_edu_duplicates > 1].count()
resume_cert_duplicates_count = resume_cert_duplicates[resume_cert_duplicates > 1].count()
resume_lang_duplicates_count = resume_lang_duplicates[resume_lang_duplicates > 1].count()

resume_edu_duplicates_count, resume_cert_duplicates_count, resume_lang_duplicates_count

(0, 3376, 47)

In [15]:
# 자격증 종류 확인
unique_certificate_contents = resume_cert['certificate_contents'].nunique()
unique_certificate_contents

1786

In [16]:
# 가장 많은 자격증을 가진 사람이 몇 개의 자격증을 가졌는지.
max_certificates = resume_cert['resume_seq'].value_counts().max()
max_certificates

10

In [17]:
# resume_seq별로 자격증 그룹화 및 개수 반영(null값 제거)
resume_cert_cleaned = resume_cert.dropna(subset=['certificate_contents'])

cert_grouped = resume_cert_cleaned.groupby('resume_seq').agg(
    certificate_list=('certificate_contents', list),
    certificate_count=('certificate_contents', 'size')
).reset_index()

all_resume_seq = resume[['resume_seq']]
cert_grouped_complete = all_resume_seq.merge(cert_grouped, on='resume_seq', how='left')
cert_grouped_complete['certificate_list'] = cert_grouped_complete['certificate_list'].apply(lambda x: x if isinstance(x, list) else [])
cert_grouped_complete['certificate_count'].fillna(0, inplace=True)
cert_grouped_complete['certificate_count'] = cert_grouped_complete['certificate_count'].astype(int)

cert_grouped_complete.head()

,resume_seq,certificate_list,certificate_count
0,U00606,[],0
1,U00509,[],0
2,U02012,[중등 2급 정교사],1
3,U04599,"[MCAS MASTER 2007, VMD MASTER]",2
4,U07573,"[컴퓨터그래픽스운용기능사, 워드프로세서 2급]",2


In [18]:
# 외국어 종류 및 자격 개수만 원핫 인코딩, 점수나 시험 종류는 드랍.
lang_encoded = pd.get_dummies(resume_lang, columns=['language'], prefix="", prefix_sep="")
lang_grouped = lang_encoded.groupby('resume_seq').sum().reset_index()
lang_grouped_cleaned = lang_grouped.drop(columns=['exam_name', 'score'])
lang_grouped_cleaned.columns = ['resume_seq', 'lang_2', 'lang_3', 'lang_4', 'lang_8', 'lang_9']
lang_grouped_cleaned[['lang_2', 'lang_3', 'lang_4', 'lang_8', 'lang_9']] = lang_grouped_cleaned[['lang_2', 'lang_3', 'lang_4', 'lang_8', 'lang_9']].clip(upper=1)
lang_grouped_cleaned

,resume_seq,lang_2,lang_3,lang_4,lang_8,lang_9
0,U00004,1,0,0,0,0
1,U00005,1,0,0,0,0
2,U00039,1,0,0,0,0
3,U00054,1,0,0,0,0
4,U00067,1,0,0,0,0
...,...,...,...,...,...,...
815,U08443,0,0,1,0,0
816,U08461,1,0,0,0,0
817,U08465,1,0,0,0,0
818,U08472,1,0,0,0,0


In [19]:
# 중복 없앤 이력서 관련 데이터프레임들 병합
final_resume = resume.merge(cert_grouped_complete, on='resume_seq', how='left')
final_resume = final_resume.merge(lang_grouped_cleaned, on='resume_seq', how='left')
final_resume = final_resume.merge(resume_edu, on='resume_seq', how='left')

final_resume

,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,...,hischool_gender,hischool_location_seq,univ_type_seq1,univ_type_seq2,univ_transfer,univ_location,univ_major,univ_sub_major,univ_major_type,univ_score
0,U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,스타일디자이너;우븐디자이너,재료·화학·섬유·의복,NaN,...,남여공학,11,5,5,0,11,NaN,NaN,4,70.0
1,U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,상품기획;MD;기획;머천다이저;머천다이징,재료·화학·섬유·의복,NaN,...,기타,0,5,5,0,5,NaN,NaN,4,90.0
2,U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,니트디자인,재료·화학·섬유·의복,NaN,...,기타,0,5,5,0,3,NaN,NaN,9,90.0
3,U04599,2020-05-13,2020-05-28,4,2012,0.0,2500.0,MD;기획MD,재료·화학·섬유·의복,NaN,...,여자학교,7,3,5,0,11,NaN,NaN,19,80.0
4,U07573,2019-07-23,2020-03-08,4,2010,1900.0,0.0,디자이너;남성복;스포츠웨어;편집디자인;코디네이터;일러스트레이터;VMD;MD,재료·화학·섬유·의복,NaN,...,여자학교,4,5,5,0,13,NaN,NaN,19,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U04890,2018-05-27,2020-08-26,5,1994,0.0,6250.0,상품기획;생산;글로벌소싱;남성복;여성복,재료·화학·섬유·의복,NaN,...,남자학교,3,5,5,0,3,NaN,NaN,3,70.0
8478,U05910,2019-09-27,2020-10-22,4,2013,2300.0,1900.0,소재;기획MD;남성복;온라인MD;디자이너,재료·화학·섬유·의복,NaN,...,여자학교,15,5,5,0,15,NaN,NaN,19,80.0
8479,U01468,2019-02-26,2020-06-23,4,2008,0.0,0.0,상품기획MD;우븐디자인,디자인,NaN,...,기타,0,5,5,0,18,NaN,NaN,9,80.0
8480,U05315,2020-03-27,2020-12-21,4,2016,0.0,0.0,여성복디자인;우븐디자인;다이마루디자인,재료·화학·섬유·의복,NaN,...,남여공학,5,5,5,0,3,NaN,NaN,9,80.0


In [20]:
recruitment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6695 entries, 0 to 6694
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   recruitment_seq    6695 non-null   object 
 1   address_seq1       6694 non-null   float64
 2   address_seq2       100 non-null    float64
 3   address_seq3       9 non-null      float64
 4   career_end         6695 non-null   int64  
 5   career_start       6695 non-null   int64  
 6   check_box_keyword  6695 non-null   object 
 7   education          6695 non-null   int64  
 8   major_task         6695 non-null   int64  
 9   qualifications     6695 non-null   int64  
 10  text_keyword       707 non-null    object 
dtypes: float64(3), int64(5), object(3)
memory usage: 575.5+ KB


In [21]:
recruitment['address_seq1'].unique()

array([ 3., 20.,  5.,  9., 11.,  1., nan, 13.,  2.])

In [22]:
# 공고 주소가 빈 데이터 확인
recruitment[recruitment['address_seq1'].isnull()]

,recruitment_seq,address_seq1,address_seq2,address_seq3,career_end,career_start,check_box_keyword,education,major_task,qualifications,text_keyword
3230,R01512,NaN,NaN,NaN,0,0,2507;2707,4,2,1,NaN


In [23]:
# 저 공고의 회사가 어떤 주소인지 확인하기 위해, 저 공고에 지원한 이력서들이 다른 어디에 지원했는지 확인
applied_resumes_check = apply_train[apply_train['recruitment_seq'] == 'R01512']
resume_seqs_for_R01512 = applied_resumes_check['resume_seq'].tolist()
other_applications = apply_train[apply_train['resume_seq'].isin(resume_seqs_for_R01512)]
other_recruitments_seq = other_applications['recruitment_seq'].unique()

addresses_for_other_recruitments = recruitment[recruitment['recruitment_seq'].isin(other_recruitments_seq)]['address_seq1']

addresses_for_other_recruitments.unique()

array([20.,  3., nan,  5.])

In [24]:
# 3, 5, 20 중 하나, 어느 주소가 가장 많은지 확인.
address_counts_recruitment = recruitment['address_seq1'].value_counts()
address_counts_recruitment.loc[[3.0, 5.0, 20.0]]

address_seq1
3.0     6490
5.0      174
20.0      17
Name: count, dtype: int64

In [25]:
# 3이 가장 많으므로, 3으로 채움
recruitment['address_seq1'] = recruitment['address_seq1'].fillna(3.0).astype(int)

In [26]:
# 다른 주소들은 nan값이 훨씬 많으므로, nan을 '정보없음'으로 함.
recruitment['address_seq2'].unique()

array([nan, 20.,  5.,  1., 13.,  3.,  2., 11.,  7.,  8.,  9., 17.])

In [27]:
recruitment['address_seq3'].unique()

array([nan, 20.,  3.,  8., 18.])

In [28]:
# recruitment의 check_box_keyword 고유값 계산
all_keywords = recruitment['check_box_keyword'].str.split(';').explode().dropna().unique()

num_unique_keywords = len(all_keywords)
num_unique_keywords

61

In [29]:
# recruitment 데이터프레임에서 text_keyword가 있으면 1, 아니면 0
recruitment['has_text_keyword'] = recruitment['text_keyword'].notna().astype(int)

# text_keyword에 해당 키워드가 있으면 1, 아니면 0
recruitment['part_time'] = recruitment['text_keyword'].str.contains('아르바이트').fillna(0).astype(int)
recruitment['intern'] = recruitment['text_keyword'].str.contains('인턴').fillna(0).astype(int)
recruitment['entry_level'] = recruitment['text_keyword'].str.contains('신입').fillna(0).astype(int)
recruitment['experienced'] = recruitment['text_keyword'].str.contains('경력|경력직').fillna(0).astype(int)
recruitment['team_leader'] = recruitment['text_keyword'].str.contains('팀장|팀장급').fillna(0).astype(int)

# check_box_keyword를 나눈 다음 one_hot encoding
check_box_encoded = recruitment['check_box_keyword'].str.get_dummies(sep=';')
recruitment = pd.concat([recruitment, check_box_encoded], axis=1)

# 원래 열들 드랍
recruitment_cleaned = recruitment.drop(columns=['address_seq2', 'address_seq3', 'text_keyword', 'check_box_keyword'])
recruitment_cleaned

,recruitment_seq,address_seq1,career_end,career_start,education,major_task,qualifications,has_text_keyword,part_time,intern,...,3005,3006,3007,3008,3099,3101,3102,3103,3105,3199
0,R02264,3,0,0,4,8,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,R06317,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,R04017,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,R02865,3,0,0,2,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,R04890,3,0,0,2,2,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,R03678,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6691,R04593,3,0,0,4,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6692,R03252,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6693,R05130,3,0,0,2,2,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# company와 recruitment 병합 전 company데이터 살펴보기.
unique_company_type_seq = company['company_type_seq'].unique()
unique_supply_kind = company['supply_kind'].unique()
unique_employee = company['employee'].unique()

unique_company_type_seq, unique_supply_kind, unique_employee

(array([2, 4, 5, 9, 7, 3]),
 array([514, 402, 100, 201, 515, 508, 501, 409, 513, 412, 601, 507, 512,
        605, 413, 502, 603]),
 array([   20,    90,    60,   590,   250,   130,    40,   200,  1000,
          500,   800,   525,   260,   150,    97,   300,   270,   100,
           41,    45,   310,   120,   180,     0,    70,     9,   145,
           35,    28,    17,     6,    10,     4,    32,    56,    15,
          390,   350,     8,    80,    30,    13,     5,   550,   631,
          650,    98,    19,    16,    22,  1600,    12,  2000,    50,
          160,     2,    25,    42, 12223,   420,   950,     3,   170,
           58,   591,    23,     7,   110,   600,   280,   118,    74,
          140,  3500,   230,   210,   400,    11,    18,   780,   750]))

In [31]:
# employee 최소, 최대
min_employee = company['employee'].min()
max_employee = company['employee'].max()

min_employee, max_employee

(0, 12223)

In [32]:
# company 정보와 최종 병합. 없는 데이터는 일단 nan으로.
final_recruitment = recruitment_cleaned.merge(company, on='recruitment_seq', how='left')
final_recruitment

,recruitment_seq,address_seq1,career_end,career_start,education,major_task,qualifications,has_text_keyword,part_time,intern,...,3008,3099,3101,3102,3103,3105,3199,company_type_seq,supply_kind,employee
0,R02264,3,0,0,4,8,1,0,0,0,...,0,0,0,0,0,0,0,5.0,402.0,800.0
1,R06317,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,R04017,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
3,R02865,3,0,0,2,2,1,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,R04890,3,0,0,2,2,2,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,R03678,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
6691,R04593,3,0,0,4,2,1,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
6692,R03252,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,4.0,402.0,525.0
6693,R05130,3,0,0,2,2,2,0,0,0,...,0,0,0,0,0,0,0,2.0,402.0,40.0


In [33]:
'''
employee nan값 채우기
'정보없음' : 0 - nan값인 경우
'영세기업' : 1 - employee가 5인 미만인 경우
'중소기업' : 2 - employee가 1000명 미만인 경우
'중견기업' : 3 - 'employee가 1000명 이상인 경우
다른 열도 nan값 채우기, '정보없음'이라는 의미로 0
'''
final_recruitment['employee_category'] = np.where(final_recruitment['employee'].isna(), 0,
                                         np.where(final_recruitment['employee'] < 5, 1,
                                                  np.where(final_recruitment['employee'] < 1000, 2, 3)))

final_recruitment = final_recruitment.drop(columns=['employee'])
final_recruitment[['company_type_seq', 'supply_kind']] = final_recruitment[['company_type_seq', 'supply_kind']].fillna(0)
final_recruitment

,recruitment_seq,address_seq1,career_end,career_start,education,major_task,qualifications,has_text_keyword,part_time,intern,...,3008,3099,3101,3102,3103,3105,3199,company_type_seq,supply_kind,employee_category
0,R02264,3,0,0,4,8,1,0,0,0,...,0,0,0,0,0,0,0,5.0,402.0,2
1,R06317,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,R04017,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,R02865,3,0,0,2,2,1,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,R04890,3,0,0,2,2,2,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,R03678,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
6691,R04593,3,0,0,4,2,1,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
6692,R03252,3,0,0,3,2,1,0,0,0,...,0,0,0,0,0,0,0,4.0,402.0,2
6693,R05130,3,0,0,2,2,2,0,0,0,...,0,0,0,0,0,0,0,2.0,402.0,2


In [34]:
# job_code_seq2와 job_code_seq3의 유의값, nan값 개수 다시 살펴보기
job_code_seq2_nan_count = final_resume['job_code_seq2'].isna().sum()
job_code_seq2_non_nan_count = final_resume['job_code_seq2'].notna().sum()

job_code_seq3_nan_count = final_resume['job_code_seq3'].isna().sum()
job_code_seq3_non_nan_count = final_resume['job_code_seq3'].notna().sum()

job_code_seq2_nan_count, job_code_seq2_non_nan_count, job_code_seq3_nan_count, job_code_seq3_non_nan_count

(7982, 500, 8295, 187)

In [35]:
# seq2, seq3의 nan을 '정보없음'으로 변경.
final_resume['job_code_seq2'].fillna('정보없음', inplace=True)
final_resume['job_code_seq3'].fillna('정보없음', inplace=True)

# 각각 label encoding
label_encoders = {}

for column in ['job_code_seq1', 'job_code_seq2', 'job_code_seq3']:
    le = LabelEncoder()
    final_resume[column] = le.fit_transform(final_resume[column])
    label_encoders[column] = le

final_resume[['job_code_seq1', 'job_code_seq2', 'job_code_seq3']].head()

,job_code_seq1,job_code_seq2,job_code_seq3
0,7,11,10
1,7,11,10
2,7,11,10
3,7,11,10
4,7,11,10


In [36]:
# language 결측치는 어차피 없는 정보이므로 모두 0으로 통일
final_resume[['lang_2', 'lang_3', 'lang_4', 'lang_8', 'lang_9']] = final_resume[['lang_2', 'lang_3', 'lang_4', 'lang_8', 'lang_9']].fillna(0)

In [37]:
# career_job_code의 경우에도 nan은 정보없음으로 하고, 나머지 label_encoding
final_resume['career_job_code'].fillna('정보없음', inplace=True)

le_career = LabelEncoder()

final_resume['career_job_code'] = le_career.fit_transform(final_resume['career_job_code'])

final_resume['career_job_code'].head()

0    35
1    21
2    39
3    21
4    35
Name: career_job_code, dtype: int64

In [38]:
# 전공은 univ_major_type으로 대체. univ_major와 univ_sub_major는 drop
final_resume = final_resume.drop(['univ_major', 'univ_sub_major'], axis=1)
final_resume

,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,...,hischool_special_type,hischool_nation,hischool_gender,hischool_location_seq,univ_type_seq1,univ_type_seq2,univ_transfer,univ_location,univ_major_type,univ_score
0,U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,스타일디자이너;우븐디자이너,7,11,...,일반고,공립,남여공학,11,5,5,0,11,4,70.0
1,U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,상품기획;MD;기획;머천다이저;머천다이징,7,11,...,기타,기타,기타,0,5,5,0,5,4,90.0
2,U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,니트디자인,7,11,...,기타,기타,기타,0,5,5,0,3,9,90.0
3,U04599,2020-05-13,2020-05-28,4,2012,0.0,2500.0,MD;기획MD,7,11,...,일반고,사립,여자학교,7,3,5,0,11,19,80.0
4,U07573,2019-07-23,2020-03-08,4,2010,1900.0,0.0,디자이너;남성복;스포츠웨어;편집디자인;코디네이터;일러스트레이터;VMD;MD,7,11,...,일반고,공립,여자학교,4,5,5,0,13,19,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U04890,2018-05-27,2020-08-26,5,1994,0.0,6250.0,상품기획;생산;글로벌소싱;남성복;여성복,7,11,...,일반고,사립,남자학교,3,5,5,0,3,3,70.0
8478,U05910,2019-09-27,2020-10-22,4,2013,2300.0,1900.0,소재;기획MD;남성복;온라인MD;디자이너,7,11,...,일반고,공립,여자학교,15,5,5,0,15,19,80.0
8479,U01468,2019-02-26,2020-06-23,4,2008,0.0,0.0,상품기획MD;우븐디자인,4,11,...,기타,기타,기타,0,5,5,0,18,9,80.0
8480,U05315,2020-03-27,2020-12-21,4,2016,0.0,0.0,여성복디자인;우븐디자인;다이마루디자인,7,11,...,일반고,공립,남여공학,5,5,5,0,3,9,80.0


In [39]:
# text_keyword에서 ;를 기준으로 분리
keywords = final_resume['text_keyword'].str.split(';').dropna().tolist()

# FastText 학습
model = FastText(sentences=keywords, vector_size=100, window=5, min_count=2, workers=4, sg=1, epochs=200)

# 임베딩 결과 확인을 위한 샘플 키워드 출력
sample_keyword = "디자이너"
model.wv.most_similar(sample_keyword)

[('디자이너신입', 0.762125551700592),
 ('디자이너상품기획', 0.6901536583900452),
 ('5년차디자이너', 0.6579981446266174),
 ('우분디자이너', 0.6576942205429077),
 ('디자이너여성복', 0.6461623311042786),
 ('디자이너인턴', 0.6417450904846191),
 ('여성복패션디자이너', 0.6394168734550476),
 ('머천다이저디자이너', 0.6334587335586548),
 ('바잉디자이너', 0.629180908203125),
 ('막내디자이너', 0.6291711330413818)]

In [40]:
# 군집화
words = list(model.wv.index_to_key)
vectors = [model.wv[word] for word in words]

n_clusters = 61 # recruitment check_box_keyword의 고유값과 같은 숫자로 매핑
kmeans = KMeans(n_clusters=n_clusters, random_state=CFG['SEED']).fit(vectors)

for cluster_num in range(n_clusters):
    words_in_cluster = [words[i] for i, label in enumerate(kmeans.labels_) if label == cluster_num]
    print(f"Cluster {cluster_num+1}: {', '.join(words_in_cluster)}")

Cluster 1: 그래픽디자인, 일러스트, 포토샵, 일러스트레이터, 그래픽, 패션그래픽, 패션그래픽디자인, 의류그래픽, 스타일링, 산업디자인, 패션일러스트, 패션광고, GRAPHIC
Cluster 2: 여성복, 캐쥬얼, 의상, 경력, 영캐주얼, 여성캐주얼, 여성, 스타일, 아동, 실장, 남성, 남성캐주얼, 대졸, 영캐쥬얼, 숙녀복, 캐릭터, 유니, 쉐터, 스타일팀, 유니섹스, 티셔츠, 프리랜서, 영캐릭터, 여성정장, 유니캐주얼, 키즈, 캐릭터캐쥬얼
Cluster 3: 유통망관리, 온라인쇼핑몰, 분석, 자사몰관리, 판매사원관리, 판매분석, 온라인몰관리, 유통망확장, 온라인유통망관리
Cluster 4: 의류디자이너, 남성복디자이너, 아동복디자이너, 아웃도어디자이너, 캐주얼웨어디자이너, 스포츠웨어디자이너, 골프웨어디자이너, 캐쥬얼디자이너, 유아동복디자이너, 스포츠디자이너, 캐쥬얼웨어디자이너, 이너웨어디자이너, 케주얼웨어디자이너, 캐릭터디자이너
Cluster 5: 자재구매, 상품관리, 자재구매관리, 원부자재구매, 부자재구매, 부자재관리, 원부자재관리, 소재관리
Cluster 6: 기술영업, 판매영업, 면세점영업, 식품영업, 법인영업, 화장품영업, 제약영업, 의료기영업, 병의원영업, 금융영업, 채널영업, 시스템영업, 광고영업, 장비영업, 솔루션영업, 식품가공, 보험영업, 인쇄영업
Cluster 7: 패션디자인, 여성복디자인, 디자인, 의상디자인, 우븐디자인, 의류디자인, 남성복디자인, 니트디자인, 잡화디자인, 아동복디자인, 가방디자인, 악세사리디자인, 이너웨어디자인, 신발디자인, 쥬얼리디자인, 용품디자인, 웹디자인, 핸드백디자인, 스타일디자인, 여성의류디자인, 색채, 제품디자인, 상품기획및디자인, 캐드, 악세서리디자인, 패션의상디자인, 상품디자인, 상품기획디자인, 색채디자인, 경력디자인, 유아복디자인, 주얼리디자인, 원단디자인, 금속디자인
Cluster 8: 경영지원, 경영기획, 경영, 경영관리, 사업기획, 경영전략, 조직관리, 경영분석, 사업제휴, 경영컨설팅, 경영

In [41]:
# 각 키워드가 어떤 클러스터에 속하는지 확인하는 함수
def get_clusters_for_keywords(keywords_list, model, kmeans):
    clusters = [kmeans.predict([model.wv[k]])[0] if k in model.wv.index_to_key else -1 for k in keywords_list]
    return clusters

# 키워드를 분리하고 각 키워드의 클러스터를 가져옴.
# NaN 값이 있을 경우 빈 리스트로 처리.
all_clusters = final_resume['text_keyword'].str.split(';').fillna('').apply(lambda x: get_clusters_for_keywords(x, model, kmeans))

# 각 클러스터에 대해 final_resume에 새로운 열을 추가.
for i in range(n_clusters):
    final_resume[f'keyword_cluster_{i+1}'] = all_clusters.apply(lambda clusters: int(i in clusters))

# text_keyword 열 삭제
final_resume.drop(columns=['text_keyword'], inplace=True)

In [42]:
# certificate의 맞춤법 교정 및 클러스터링을 시도하려고 했으나, 제대로 클러스터링이 안되는 것 같음. 
# recruitment의 qualification에 해당되는 지원자만 지원했다고 생각하고, certificate_list는 드랍.
# 단 certificate_count는 남겨놓음.

final_resume = final_resume.drop('certificate_list', axis=1)
final_resume

,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,job_code_seq1,job_code_seq2,job_code_seq3,...,keyword_cluster_52,keyword_cluster_53,keyword_cluster_54,keyword_cluster_55,keyword_cluster_56,keyword_cluster_57,keyword_cluster_58,keyword_cluster_59,keyword_cluster_60,keyword_cluster_61
0,U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,7,11,10,...,0,0,0,0,0,1,0,0,0,0
1,U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,7,11,10,...,0,0,0,0,0,0,0,0,0,0
2,U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,7,11,10,...,0,0,0,0,0,0,0,0,0,0
3,U04599,2020-05-13,2020-05-28,4,2012,0.0,2500.0,7,11,10,...,0,0,0,0,0,0,0,0,0,0
4,U07573,2019-07-23,2020-03-08,4,2010,1900.0,0.0,7,11,10,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U04890,2018-05-27,2020-08-26,5,1994,0.0,6250.0,7,11,10,...,0,0,0,0,0,0,0,0,0,0
8478,U05910,2019-09-27,2020-10-22,4,2013,2300.0,1900.0,7,11,10,...,0,0,0,0,0,1,0,0,0,0
8479,U01468,2019-02-26,2020-06-23,4,2008,0.0,0.0,4,11,10,...,0,0,0,0,0,0,0,0,0,0
8480,U05315,2020-03-27,2020-12-21,4,2016,0.0,0.0,7,11,10,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# 아직 object로 남아있는 부분 정리
# 1. resume_edu에서 가져왔던 고등학교 정보들 label_encoding
columns_to_encode = ['hischool_special_type', 'hischool_nation', 'hischool_gender']

for col in columns_to_encode:
    le = LabelEncoder()
    final_resume[col] = le.fit_transform(final_resume[col])

In [44]:
# 시간 정보 변환
final_resume['updated_date'] = pd.to_datetime(final_resume['updated_date'])
final_resume['reg_date'] = pd.to_datetime(final_resume['reg_date'])

final_resume

,resume_seq,degree,graduate_date,hope_salary,last_salary,job_code_seq1,job_code_seq2,job_code_seq3,career_month,career_job_code,...,keyword_cluster_54,keyword_cluster_55,keyword_cluster_56,keyword_cluster_57,keyword_cluster_58,keyword_cluster_59,keyword_cluster_60,keyword_cluster_61,update_reg_diff,current_update_diff
0,U00606,4,2008,3500.0,3500.0,7,11,10,67,35,...,0,0,0,1,0,0,0,0,79,1258
1,U00509,2,0,0.0,3700.0,7,11,10,84,21,...,0,0,0,0,0,0,0,0,374,1155
2,U02012,5,1979,3500.0,3100.0,7,11,10,121,39,...,0,0,0,0,0,0,0,0,797,1375
3,U04599,4,2012,0.0,2500.0,7,11,10,24,21,...,0,0,0,0,0,0,0,0,15,1252
4,U07573,4,2010,1900.0,0.0,7,11,10,0,35,...,0,0,0,1,0,1,0,0,229,1333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U04890,5,1994,0.0,6250.0,7,11,10,204,21,...,0,0,0,0,0,0,0,0,822,1162
8478,U05910,4,2013,2300.0,1900.0,7,11,10,12,21,...,0,0,0,1,0,0,0,0,391,1105
8479,U01468,4,2008,0.0,0.0,4,11,10,0,21,...,0,0,0,0,0,0,0,0,483,1226
8480,U05315,4,2016,0.0,0.0,7,11,10,18,13,...,0,0,0,0,0,0,0,0,269,1045


In [45]:
# graduate_date가 0인 데이터가 많은데, degree로 확인이 불가하므로 그냥 0으로 둠.
final_resume[final_resume['graduate_date'] == 0]

,resume_seq,degree,graduate_date,hope_salary,last_salary,job_code_seq1,job_code_seq2,job_code_seq3,career_month,career_job_code,...,keyword_cluster_54,keyword_cluster_55,keyword_cluster_56,keyword_cluster_57,keyword_cluster_58,keyword_cluster_59,keyword_cluster_60,keyword_cluster_61,update_reg_diff,current_update_diff
1,U00509,2,0,0.0,3700.0,7,11,10,84,21,...,0,0,0,0,0,0,0,0,374,1155
6,U04073,3,0,3900.0,3900.0,7,11,10,77,21,...,0,0,0,0,0,0,0,0,1304,1113
24,U06979,4,0,0.0,3500.0,2,9,7,50,12,...,0,1,0,0,0,0,0,0,1,1063
25,U04565,2,0,2900.0,2900.0,7,11,10,72,21,...,0,1,0,0,0,0,0,0,14,1228
29,U00198,4,0,2700.0,2700.0,7,11,10,6,21,...,0,0,0,0,0,0,0,0,26,1067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8443,U06133,4,0,0.0,0.0,7,11,10,0,35,...,0,0,0,0,0,0,0,0,22,1385
8445,U00524,4,0,2300.0,0.0,7,11,10,0,35,...,0,0,0,0,0,0,0,0,95,1328
8449,U04682,4,0,0.0,5750.0,7,11,10,219,12,...,0,1,0,0,0,0,0,0,1374,1090
8454,U05013,4,0,0.0,2100.0,7,11,10,60,21,...,0,0,0,0,0,0,0,0,1666,1078


In [60]:
# graphSAGE 모델 만들기
# 1. bi-partite 그래프 형성.

# ID 매핑
resume_ids = {v: k for k, v in enumerate(final_resume['resume_seq'].unique())}
recruitment_ids = {v: k for k, v in enumerate(final_recruitment['recruitment_seq'].unique())}

# 엣지 생성
edges = apply_train.apply(lambda row: (resume_ids[row['resume_seq']], recruitment_ids[row['recruitment_seq']]), axis=1)
# 랜덤 워크 위한 양방향 관계를 추가.
edges_reversed = [(dst, src) for src, dst in zip(src_nodes, dst_nodes)]

# 소스와 목적지 노드 리스트
src_nodes = [edge[0] for edge in edges]
dst_nodes = [edge[1] for edge in edges]

# 소스와 목적지 노드 리스트 (역방향)
src_nodes_reversed = [edge[0] for edge in edges_reversed]
dst_nodes_reversed = [edge[1] for edge in edges_reversed]

# 이분 그래프 생성
graph_data = {
    ('resume', 'applies_to', 'recruitment'): (torch.tensor(src_nodes), torch.tensor(dst_nodes)),
    ('recruitment', 'is_applied_by', 'resume'): (torch.tensor(src_nodes_reversed), torch.tensor(dst_nodes_reversed))
}
G = dgl.heterograph(graph_data)

# 특성 스케일링
scaler_resume = MinMaxScaler()
scaler_recruitment = MinMaxScaler()

resume_features_scaled = scaler_resume.fit_transform(final_resume.drop(columns=['resume_seq']).values)
recruitment_features_scaled = scaler_recruitment.fit_transform(final_recruitment.drop(columns=['recruitment_seq']).values)

resume_features_tensor = torch.tensor(resume_features_scaled, dtype=torch.float32)
recruitment_features_tensor = torch.tensor(recruitment_features_scaled, dtype=torch.float32)

# 이분 그래프에 특성 설정
G.nodes['resume'].data['features'] = resume_features_tensor
G.nodes['recruitment'].data['features'] = recruitment_features_tensor

# 그래프 정보 출력
num_resumes = G.num_nodes('resume')
num_recruitments = G.num_nodes('recruitment')
num_edges = G.num_edges('applies_to')

print("Number of resumes:", num_resumes)
print("Number of recruitments:", num_recruitments)
print("Number of edges:", num_edges)

Number of resumes: 8482
Number of recruitments: 6695
Number of edges: 57946


In [57]:
class ItemToItemBatchSampler(IterableDataset):
    def __init__(self, g, resume_type, recruitment_type, batch_size):
        self.g = g
        self.resume_type = resume_type
        self.recruitment_type = recruitment_type
        self.resume_to_recruitment_etype = ('resume', 'applies_to', 'recruitment')
        self.recruitment_to_resume_etype = ('recruitment', 'is_applied_by', 'resume')
        self.batch_size = batch_size

    def __iter__(self):
        while True:
            heads = torch.randint(0, self.g.number_of_nodes(self.resume_type), (self.batch_size,))
            result = dgl.sampling.random_walk(
                self.g,
                heads,
                metapath=[self.resume_to_recruitment_etype, self.recruitment_to_resume_etype])
            tails = result[0][:, 2]
            neg_tails = torch.randint(0, self.g.number_of_nodes(self.recruitment_type), (self.batch_size,))
            mask = (tails != -1)
            yield heads[mask], tails[mask], neg_tails[mask]

In [58]:
# sampling : 코드 참고(https://yamalab.tistory.com/165)
class NeighborSampler(object):
    def __init__(self, g, resume_type, recruitment_type, random_walk_length, random_walk_restart_prob,
                 num_random_walks, num_neighbors, num_layers):
        self.g = g
        self.resume_type = resume_type
        self.recruitment_type = recruitment_type
        self.resume_to_recruitment_etype = ('resume', 'applies_to', 'recruitment')
        self.recruitment_to_resume_etype = ('recruitment', 'is_applied_by', 'resume')
        self.samplers = [
            dgl.sampling.PinSAGESampler(g, recruitment_type, resume_type, random_walk_length,
                                        random_walk_restart_prob, num_random_walks, num_neighbors)
            for _ in range(num_layers)]

    def sample_blocks(self, seeds, heads=None, tails=None, neg_tails=None):
        blocks = []
        for sampler in self.samplers:
            frontier = sampler(seeds)
            block = compact_and_copy(frontier, seeds)
            seeds = block.srcdata[dgl.NID]
            blocks.insert(0, block) 
        return blocks

    def sample_from_item_pairs(self, heads, tails, neg_tails):
        pos_graph = dgl.graph(
            (heads, tails),
            num_nodes=self.g.number_of_nodes(self.recruitment_type))
        neg_graph = dgl.graph(
            (heads, neg_tails),
            num_nodes=self.g.number_of_nodes(self.recruitment_type))
        pos_graph, neg_graph = dgl.compact_graphs([pos_graph, neg_graph])
        seeds = pos_graph.ndata[dgl.NID]
        blocks = self.sample_blocks(seeds, heads, tails, neg_tails)
        return pos_graph, neg_graph, blocks

In [ ]:
class PinSAGECollator(object):
    def __init__(self, sampler, g, ntype):
        self.sampler = sampler
        self.ntype = ntype
        self.g = g

    def collate_train(self, batches):
        heads, tails, neg_tails = batches[0]
        pos_graph, neg_graph, blocks = self.sampler.sample_from_item_pairs(heads, tails, neg_tails)
        assign_features_to_blocks(blocks, self.g, self.ntype)
        return pos_graph, neg_graph, blocks

    def collate_valid(self, batches):
        heads, tails = batches[0], batches[1]
        pos_graph = dgl.graph((heads, tails), num_nodes=self.g.number_of_nodes(self.ntype))
        pos_graph = dgl.compact_graphs([pos_graph])[0]
        seeds = pos_graph.ndata[dgl.NID]
        blocks = self.sampler.sample_blocks(seeds)
        assign_features_to_blocks(blocks, self.g, self.ntype)
        return pos_graph, blocks